Dimer with Density-Density interaction and Spin-Orbit Coupling
=======

We calculate the Green function of a dimer coupled with Kanamori interaction and a coupling to discrete bath states. The spin block-structure is broken by a spin-orbit term in the local Hamiltonian.

The non-interacting Hamiltonian is most conveniently expressed using a matrix representation

$$
\hat{h} =
\begin{pmatrix}
\hat{h}_{0} & \hat{h}_{\rm coup} \\
\hat{h}_{\rm coup} & \hat{h}_{\rm bath}  \\
\end{pmatrix}=
\begin{pmatrix}
\begin{pmatrix}
\epsilon_0 -\mu & -t - ia & 0 & -ia\\
-t + ia& \epsilon_1 -\mu & ia & 0\\
0 & - ia & \epsilon_0 -\mu & -t - ia\\
ia & 0 & -t + ia& \epsilon_1 -\mu
\end{pmatrix} & 
\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{pmatrix}
\\
\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{pmatrix} & 
\begin{pmatrix}
\epsilon_{0,{\rm bath}} & -t_{\rm bath} & 0 & 0 \\
-t_{\rm bath} & \epsilon_{1,{\rm bath}} & 0 & 0 \\
0 & 0 & \epsilon_{0,{\rm bath}} & -t_{\rm bath}\\
0 & 0 & -t_{\rm bath} & \epsilon_{1,{\rm bath}}
\end{pmatrix}
\end{pmatrix}
$$

The impurity interaction is of the density-density type, and reads

$$h_{\rm int} = \sum_{i,j} U n_{\uparrow,i} n_{\downarrow,j} + \sum_{\sigma,i\neq j} U' n_{\sigma,i} n_{\sigma,j}$$
   
The parameters are defined below.

In [ ]:
# %load model.py
import sys, os
sys.path.append(os.getcwd() + '/../common')
from util import *

from pytriqs.gf import Gf, MeshImFreq, iOmega_n, inverse
from pytriqs.operators import c, c_dag, n
from pytriqs.operators.util.hamiltonians import h_int_kanamori
from itertools import product
from numpy import matrix, array, diag

# ==== System Parameters ====
beta = 5.                       # Inverse temperature
mu = 0.25                       # Chemical potential
eps = array([0.0, 0.1, 0.0, 0.1])   # Impurity site energies
t = 1.                          # Hopping between impurity sites
a = 1j                          # Spin-orbit coupling

eps_bath = array([0.2, 0.15, 0.2, 0.15])   # Bath site energies
t_bath = 0.1                    # Hopping between bath sites

U = 1.                          # Density-density interaction for opposite spins
Up = 0.3                        # Density-density interaction for equal spins

up_0, up_1, dn_0, dn_1 = 0, 1, 2, 3
orb_names  = [up_0, up_1, dn_0, dn_1]

# Non-interacting impurity hamiltonian in matrix representation
h_0_mat = diag(eps - mu) - matrix([[0,    t+a,  0,    a  ],
                                   [t-a,  0,   -a,    0  ],
                                   [0,    a,    0,    t+a],
                                   [-a,    0,    t-a,  0  ]])

# Bath hamiltonian in matrix representation
h_bath_mat = diag(eps_bath) - matrix([[0,       t_bath, 0,      0     ],
                                      [t_bath,  0,      0,      0     ],
                                      [0,       0,      0,      t_bath],
                                      [0,       0,      t_bath, 0     ]])

# Coupling matrix
V_mat = matrix([[1., 0., 0, 0],
                [0., 1., 0, 0],
                [0., 0., 1, 0],
                [0., 0., 0, 1]])

# ==== Local Hamiltonian ====
c_dag_vec = matrix([[c_dag('bl',o) for o in orb_names]])
c_vec = matrix([[c('bl',o)] for o in orb_names])

h_0 = (c_dag_vec * h_0_mat * c_vec)[0,0]

h_int = U * n('bl', up_0) * n('bl', dn_0) + \
        U * n('bl', up_1) * n('bl', dn_1) + \
        U * n('bl', up_0) * n('bl', dn_1) + \
        U * n('bl', up_1) * n('bl', dn_0) + \
        Up * n('bl', up_0) * n('bl', up_1) + \
        Up * n('bl', dn_0) * n('bl', dn_1)

h_loc = h_0 + h_int

# ==== Bath & Coupling hamiltonian ====
orb_bath_names = ['b_' + str(o) for o in orb_names]
c_dag_bath_vec = matrix([[c_dag('bl', o) for o in orb_bath_names]])
c_bath_vec =     matrix([[c('bl', o)] for o in orb_bath_names])

h_bath = (c_dag_bath_vec * h_bath_mat * c_bath_vec)[0,0]
h_coup = (c_dag_vec * V_mat * c_bath_vec + c_dag_bath_vec * V_mat * c_vec)[0,0] # FIXME Adjoint

# ==== Total impurity hamiltonian ====
h_imp = h_loc + h_coup + h_bath

# ==== Green function structure ====
gf_struct = [ ['bl', orb_names] ]

# ==== Hybridization Function ====
n_iw = 10
iw_mesh = MeshImFreq(beta, 'Fermion', n_iw)
Delta = BlockGf(mesh=iw_mesh, gf_struct=gf_struct)
Delta << inverse(iOmega_n - V_mat * h_bath_mat * V_mat);

# ==== Non-Interacting Impurity Green function  ====
G0_iw = Delta.copy()
G0_iw['bl'] << inverse(iOmega_n - h_0_mat - Delta['bl'])


Results
====

In [ ]:
from pytriqs.plot.mpl_interface import oplot, plt
%matplotlib inline

In [ ]:
# %load ../common/plot.py
import sys, os
sys.path.append(os.getcwd() + "/..")
sys.path.append(os.getcwd() + "/../../common")
from model import *

from pytriqs.archive import HDFArchive
from pytriqs.plot.mpl_interface import oplot, plt
from glob import glob
from os.path import basename

# === Load Green function for every solver and calculate self-energy

solver_lst = [ basename(f).strip('.h5') for f in glob('results/*.h5') ]
G, Sigma = {}, {}

for solver in solver_lst:
    dat = HDFArchive('results/' + solver + '.h5','r')
    G[solver] = dat['G']
    Sigma[solver] = G0_iw.copy()
    Sigma[solver] << inverse(G0_iw) - inverse(G[solver])

# === For every block and solver, plot Green function and Self energy

block_lst = list(G[solver_lst[0]].indices)
n_blocks = len(block_lst)

for g, name in [[G, 'G'], [Sigma, '$\Sigma$']]:

    plt.subplots(n_blocks,1,figsize=(10,6*n_blocks))

    for i, block in enumerate(block_lst,1):
        fig = plt.subplot(n_blocks,1,i)
        fig.set_title(name + "[" + block + "]")
        for solver in solver_lst:
            oplot(g[solver][block][0,1], name = name + "[0,1]_%s" % solver)
        plt.xlabel("$\omega_n$")
        plt.ylabel(name + "[" + block + "]$(i\omega_n)$")

    plt.tight_layout()
    plt.show()


Deviations
======

We present a table containing deviations between the different solvers measured via

$$||G_{\rm Solver_1} - G_{\rm Solver_2}||_\infty$$

In [ ]:
import numpy as np

for block in block_lst:
    deviations = [[ np.amax(np.abs(G[s1][block].data - G[s2][block].data)) for s1 in solver_lst ] \
                    for s2 in solver_lst ]
    
    print "\t\t    Deviations for Block " + block
    print "\t\t -----------------------------------"

    row_format ="{:>15}" * (len(solver_lst) + 1)
    print row_format.format("", *solver_lst)
    row_format ="{:>15}" + "{:>15.2E}" * len(solver_lst)
    for solver, row in zip(solver_lst, deviations):
        print row_format.format(solver, *row)
        
    print "\n\n"